In [1]:
from sqlalchemy import create_engine, text
import pandas as pd

## **Conexión a PostgreSQL**

In [2]:
# Credenciales de conexión
db_user = "etl25"
db_pass = "etl25"
db_host = "localhost"
db_port = "5433"
db_name = "dw_icfes"

# Crear conexión
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}")

# Probar conexión
with engine.connect() as conn:
    result = conn.execute(text("SELECT version();"))
    print("Conexión exitosa a PostgreSQL")
    print(result.scalar())

Conexión exitosa a PostgreSQL
PostgreSQL 16.10 (Debian 16.10-1.pgdg13+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 14.2.0-19) 14.2.0, 64-bit


## **Creación del Esquema**

In [10]:
# --- Ruta del archivo SQL ---
schema_path = "../dw/schema.sql"

# --- Leer el contenido del archivo SQL ---
with open(schema_path, "r", encoding="utf-8") as f:
    schema_sql = f.read()

# --- Ejecutar el script en la base de datos ---
with engine.connect() as conn:
    conn.execute(text(schema_sql))
    conn.commit()

print("Esquema del Data Warehouse creado exitosamente.")

Esquema del Data Warehouse creado exitosamente.


## **Insersión de Datos**

In [4]:
# === Cargar dataset final desde curated ===
df = pd.read_csv("../data/curated/df_icfes_2024_final.csv")

print("Dataset cargado correctamente:", df.shape)
df.head(5)


Dataset cargado correctamente: (626339, 39)


,periodo,estu_estudiante,cole_area_ubicacion,cole_bilingue,cole_calendario,cole_naturaleza,cole_depto_ubicacion,cole_mcpio_ubicacion,estu_depto_presentacion,estu_mcpio_presentacion,...,percentil_ingles,percentil_lectura_critica,percentil_matematicas,percentil_sociales_ciudadanas,punt_c_naturales,punt_ingles,punt_lectura_critica,punt_matematicas,punt_sociales_ciudadanas,punt_global
0,20241,ESTUDIANTE,URBANO,1,B,NO OFICIAL,"BOGOTÁ, D.C.",BOGOTA D.C.,"BOGOTÁ, D.C.",BOGOTA D.C.,...,89,78,66,91,64,84,69,65,72,344
1,20241,ESTUDIANTE,URBANO,1,A,NO OFICIAL,"BOGOTÁ, D.C.",BOGOTA D.C.,"BOGOTÁ, D.C.",BOGOTA D.C.,...,23,45,47,24,43,47,58,57,43,250
2,20241,ESTUDIANTE,URBANO,1,B,NO OFICIAL,"BOGOTÁ, D.C.",BOGOTA D.C.,"BOGOTÁ, D.C.",BOGOTA D.C.,...,44,23,19,6,53,61,49,43,33,229
3,20241,ESTUDIANTE,URBANO,0,B,NO OFICIAL,VALLE DEL CAUCA,CALI,VALLE DEL CAUCA,CALI,...,51,28,39,57,63,66,51,53,59,286
4,20241,ESTUDIANTE,URBANO,1,B,NO OFICIAL,"BOGOTÁ, D.C.",BOGOTA D.C.,"BOGOTÁ, D.C.",BOGOTA D.C.,...,63,46,36,50,55,74,59,52,56,285


In [11]:
# === 1. dim_departamento ===
dim_departamento = (
    df[["estu_depto_reside", "poblacion_depto", "idh_depto", "pobreza_monetaria_depto"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
print("Departamentos:", dim_departamento.shape)

# === 2. dim_municipio ===
dim_municipio = (
    df[["estu_mcpio_reside", "poblacion_mcpio"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
print("Municipios:", dim_municipio.shape)

# === 3. dim_contexto_socioeconomico ===
dim_contexto = (
    df[["estu_inse_individual", "estu_nse_individual", "fami_estratovivienda"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
print("Contexto socioeconómico:", dim_contexto.shape)

# === 4. dim_colegio ===
dim_colegio = (
    df[[
        "cole_area_ubicacion", "cole_bilingue", "cole_calendario", "cole_naturaleza",
        "cole_depto_ubicacion", "estu_nse_establecimiento"
    ]]
    .drop_duplicates()
    .reset_index(drop=True)
)
print("Colegios:", dim_colegio.shape)

# === 5. dim_fecha ===
dim_fecha = (
    df[["periodo"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
print("Fechas:", dim_fecha.shape)


Departamentos: (34, 4)
Municipios: (1110, 2)
Contexto socioeconómico: (598867, 3)
Colegios: (632, 6)
Fechas: (2, 1)


In [12]:
# === Paso 3: Insertar dimensiones en PostgreSQL ===
dim_departamento.to_sql("dim_departamento", con=engine, if_exists="append", index=False)
dim_municipio.to_sql("dim_municipio", con=engine, if_exists="append", index=False)
dim_contexto.to_sql("dim_contexto_socioeconomico", con=engine, if_exists="append", index=False)
dim_colegio.to_sql("dim_colegio", con=engine, if_exists="append", index=False)
dim_fecha.to_sql("dim_fecha", con=engine, if_exists="append", index=False)

print("Dimensiones insertadas exitosamente en PostgreSQL.")


Dimensiones insertadas exitosamente en PostgreSQL.


**TABLA DE HECHOS**

In [13]:
# === Paso 4: Construir tabla de hechos fact_icfes ===

# --- Unir con IDs de las dimensiones (solo claves necesarias) ---
fact_icfes = df.merge(dim_departamento.reset_index().rename(columns={"index": "id_departamento"}), 
                        on=["estu_depto_reside", "poblacion_depto", "idh_depto", "pobreza_monetaria_depto"], 
                        how="left")

fact_icfes = fact_icfes.merge(dim_municipio.reset_index().rename(columns={"index": "id_municipio"}), 
                                on=["estu_mcpio_reside", "poblacion_mcpio"], 
                                how="left")

fact_icfes = fact_icfes.merge(dim_contexto.reset_index().rename(columns={"index": "id_contexto"}), 
                                on=["estu_inse_individual", "estu_nse_individual", "fami_estratovivienda"], 
                                how="left")

fact_icfes = fact_icfes.merge(dim_colegio.reset_index().rename(columns={"index": "id_colegio"}), 
                                on=["cole_area_ubicacion", "cole_bilingue", "cole_calendario", 
                                    "cole_naturaleza", "cole_depto_ubicacion", 
                                    "estu_nse_establecimiento"], 
                                how="left")

fact_icfes = fact_icfes.merge(dim_fecha.reset_index().rename(columns={"index": "id_fecha"}), 
                                on="periodo", how="left")



In [17]:
# --- Seleccionar solo métricas y claves foráneas ---
fact_icfes = fact_icfes[[
    "id_departamento", "id_municipio", "id_contexto", "id_colegio", "id_fecha",
    "percentil_c_naturales", "percentil_global", "percentil_ingles",
    "percentil_lectura_critica", "percentil_matematicas", "percentil_sociales_ciudadanas",
    "punt_c_naturales", "punt_ingles", "punt_lectura_critica",
    "punt_matematicas", "punt_sociales_ciudadanas", "punt_global"
]]

print("Fact table shape:", fact_icfes.shape)
fact_icfes.head(3)



Fact table shape: (626339, 17)


,id_departamento,id_municipio,id_contexto,id_colegio,id_fecha,percentil_c_naturales,percentil_global,percentil_ingles,percentil_lectura_critica,percentil_matematicas,percentil_sociales_ciudadanas,punt_c_naturales,punt_ingles,punt_lectura_critica,punt_matematicas,punt_sociales_ciudadanas,punt_global
0,1,1,1,1,1,71,79,89,78,66,91,64,84,69,65,72,344
1,1,1,2,2,1,22,32,23,45,47,24,43,47,58,57,43,250
2,1,1,3,3,1,42,23,44,23,19,6,53,61,49,43,33,229


In [16]:
# --- Ajustar índices para que coincidan con las FK en PostgreSQL ---
fact_icfes[["id_departamento", "id_municipio", "id_contexto", "id_colegio", "id_fecha"]] += 1


In [18]:
fact_icfes.to_sql("fact_icfes", con=engine, if_exists="append", index=False)
print("Tabla de hechos cargada exitosamente en PostgreSQL.")


Tabla de hechos cargada exitosamente en PostgreSQL.


Granularidad de la fact table (fact_icfes)
Cada registro de la tabla de hechos representa el desempeño académico individual de un estudiante en una única presentación del examen ICFES Saber 11, en un periodo determinado.

Este nivel de detalle permite analizar los resultados a nivel:

de estudiante (micro),

de institución educativa (agregando por colegio),

de municipio o departamento,

y de año o periodo de presentación.

Las medidas asociadas son los puntajes y percentiles por área, y las dimensiones relacionadas son: Departamento, Municipio, Colegio, Contexto Socioeconómico y Fecha.